In [ ]:
import pandas as pd
from pathlib import Path
import numpy as np
import re
from tqdm.notebook import tqdm
import numpy as np
from chempy.chemistry import Substance
from rdkit.Chem import Descriptors, PeriodicTable, MolFromSmiles, MolFromInchi
from rdkit.Chem.rdchem import GetPeriodicTable
from rdkit.Chem.rdMolDescriptors import CalcMolFormula
from rdkit.Chem.MolStandardize import rdMolStandardize
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.info')

## Load Databases (HMDB, KEGG compounds, Lipid Maps Structure Database, and ChEBI)

In [ ]:
HMDB_pkl = 'HMDB/hmdb_metabolites(17_11_2021).pkl'
KEGG_pkl = 'KEGG/KEGG_compounds(20_01_2022).pkl'
LMSD_pkl = 'LMSD/LMSD_05_05_2022.pkl'
ChEBI_pkl = 'ChEBI/ChEBI_complete_04_04_2022.pkl'

In [ ]:
HMDB = pd.read_pickle(HMDB_pkl).reset_index()
KEGG = pd.read_pickle(KEGG_pkl).reset_index()
LMSD = pd.read_pickle(LMSD_pkl).reset_index()
ChEBI = pd.read_pickle(ChEBI_pkl).reset_index()

print('----------------- HMDB Dataset -----------------')
#Filter out HMDB Compounds with no InChI - no SMILES info
print('Compounds without InChI: ', end='')
n=len(HMDB)
HMDB = HMDB[HMDB['InChI'].notna()][['HMDB ID', 'InChI', 'InChIKey', 'SMILES']].reset_index(drop=True)
print(n - len(HMDB), '\n',n, '-->', len(HMDB))
HMDB.info()

print('\n----------------- KEGG Dataset -----------------')
#Filter out KEGG Compounds with no InChI - some polymers were converted, others couldn't
print('Compounds without InChI: ', end='')
n=len(KEGG)
KEGG = KEGG[KEGG['InChI'].notna()][['KEGG ID', 'InChI', 'InChIKey']].reset_index(drop=True)
print(n - len(KEGG), '\n',n, '-->', len(KEGG))
KEGG.info()

print('\n-------------- Lipid Maps Dataset --------------')
print('Compounds without InChI: ', end='')
n=len(LMSD)
LMSD = LMSD[LMSD['InChI'].notna()][['Lipid Maps ID', 'InChI', 'InChIKey', 'SMILES']].reset_index(drop=True)
print(n - len(LMSD), '\n',n, '-->', len(LMSD))
LMSD.info()

print('\n----------------- ChEBI Dataset ----------------')
#Filter out ChEBI Compounds with no InChI - no SDF, corrupted SDF or Radicals/Polymers
print('Compounds without InChI: ', end='')
n=len(ChEBI)
ChEBI = ChEBI[ChEBI['InChI'].notna()][['ChEBI ID', 'InChI', 'InChIKey', 'SMILES']].reset_index(drop=True)
print(n - len(ChEBI), '\n',n, '-->', len(ChEBI))
ChEBI.info()

## Merge all Databases

In [ ]:
HMDB = HMDB.drop_duplicates('InChI').reset_index(drop=True)
KEGG = KEGG.drop_duplicates('InChI').reset_index(drop=True)
LMSD = LMSD.drop_duplicates('InChI').reset_index(drop=True)
ChEBI = ChEBI.drop_duplicates('InChI').reset_index(drop=True)
#Inchi duplicates:
#HMDB seems to have bad annotation of the InChI (4 duplicates)
#KEGG seems to have polymers with different number of monomers - some with defined number of monomers other are not;
#and repeated compounds(85 repeated InChIs)
#LMSD Stereoisomers (4 duplicates) 
#ChEBI seems to have tautomers of the same compound or to be the same compound (1005 repeated)

In [ ]:
#Join the 4 databases by the InChIKey
all_db = HMDB.merge(LMSD, on='InChIKey', how='outer', suffixes=('_H', '_L')).merge(KEGG, on='InChIKey', how='outer').merge(ChEBI, on='InChIKey', how='outer', suffixes=('_K', '_C')).drop_duplicates('InChIKey')

#Join Inchi columns
all_db = all_db.rename(columns={'InChI_H':'InChI'})
all_db['InChI'] = all_db['InChI'].fillna(all_db['InChI_K']).fillna(all_db['InChI_L']).fillna(all_db['InChI_C'])
all_db = all_db.drop(columns=['InChI_K', 'InChI_L', 'InChI_C'])

#Join SMILES columns
all_db = all_db.rename(columns={'SMILES':'SMILES_C', 'SMILES_H':'SMILES'})
all_db['SMILES'] = all_db['SMILES'].fillna(all_db['SMILES_C']).fillna(all_db['SMILES_L'])
all_db = all_db.drop(columns=['SMILES_C', 'SMILES_L'])
all_db.to_pickle('AllDB_w_IDs.pkl')
all_db.info()

From repeated compounds with different levels of stereochemical/isotopic information or different protonation states, 
leave only one, preferably unprotonated ones<br>
InChIKey format -> AAAAAAAAAAAAAA-BBBBBBBBFV-P<br>
A -> Molecular Skeleton <br>
B -> Stereochemistry and Isotopic Substitution Information<br>
F -> Indicates the kind of InChIKey (S-Standard or N-Non-Standard)<br>
V -> Indicates the version of the InChIKey (A-version 1)<br>
P -> Protonation State<br>
In this Dataset, F and V are always equal between rows (All Standard, version 1 InChI Keys)

In [ ]:
inchikey_splited = all_db['InChIKey'].str.split('-', expand=True)
all_db['InChIKey_Skeleton'] = inchikey_splited[0]
all_db['InChIKey_unprotonated'] = [True if state == 'N' else False for state in list(inchikey_splited[2])]

#Preferentially leave uprotonated compounds
all_db.sort_values('InChIKey_unprotonated', ascending=False, inplace=True)

all_db = all_db.drop_duplicates('InChIKey_Skeleton')
all_db = all_db.reset_index(drop=True)
all_db.drop(columns=['HMDB ID', 'Lipid Maps ID', 'KEGG ID', 'ChEBI ID', 'InChIKey_Skeleton', 
                     'InChIKey_unprotonated'], inplace=True)
all_db.info()

Use RDKit to load molecules from InChI, or SMILE if failed. Neutralize (Uncharger) the molecule by adding/removing H+ and retrieve their chemical formula

In [ ]:
uncharger = rdMolStandardize.Uncharger()
chemical_form_l = []
mass_l = []
mapper = {z:GetPeriodicTable().GetElementSymbol(z) for z in range(1, 118)}
mapper[0] = 'Charge'
for i, row in tqdm(all_db[['InChI', 'SMILES']].iterrows(), total=len(all_db)):
    try:
        mol = MolFromInchi('InChI=' + row['InChI'])
    except:
        mol = None
    if mol is None:
        print('InChI read failed')
        if not pd.isna(row['SMILES']):
            mol = MolFromSmiles(row['SMILES'])
            if mol is None:
                print('SMILES read failed')
                print(row)
                chemical_form_l.append({})
                mass_l.append({})
                continue
        else:
            print('No SMILES info')
            print(row)
            chemical_form_l.append({})
            mass_l.append({})
            continue
    mol = uncharger.uncharge(mol)
    chemical_form = CalcMolFormula(mol)
    chem_count = Substance.from_formula(chemical_form).composition
    chem_count = {mapper[z]: (chem_count[z] if z in chem_count else 0) for z in mapper} #convert z to elem name and add 0's to the rest of the elements
    chem_count['Chemical Formula'] = chemical_form
    chemical_form_l.append(chem_count)
    mass_l.append({'Mass': Descriptors.ExactMolWt(mol)})
    
all_db = pd.concat([all_db, pd.DataFrame(chemical_form_l), pd.DataFrame(mass_l)], axis=1)
all_db = all_db.rename(columns=mapper)
all_db.info()

In [ ]:
all_db[all_db['Chemical Formula'].isna()]

RDKit failed to retrive the Chemical Formula of 42 compounds, which will be annotated directly using the formula in the InChI string 

In [ ]:
mapper = {z:GetPeriodicTable().GetElementSymbol(z) for z in range(1, 118)}
mapper[0] = 'Charge'

for i, row in all_db[all_db['Chemical Formula'].isna()].iterrows():
    formula = row['InChI'].split('/')[1]
    all_db['Chemical Formula'].loc[i] = formula
    
    #compute elemental composition
    chem_count = Substance.from_formula(formula).composition
    for elem in mapper:
        key = mapper[elem]
        if elem in chem_count.keys():
            all_db[key].loc[i] = int(chem_count[elem])
        else:
            all_db[key].loc[i] = 0
    #calculate masses
    parts = re.findall("[A-Z][a-z]?|[0-9]+", formula)
    monois_mass = 0
    
    for index in range(len(parts)):
        if parts[index].isnumeric():
            continue
        
        multiplier = int(parts[index + 1]) if len(parts) > index + 1 and parts[index + 1].isnumeric() else 1
        monois_mass += PeriodicTable.GetMostCommonIsotopeMass(GetPeriodicTable(), parts[index]) * multiplier
    all_db.loc[i, 'Mass'] = monois_mass
    
all_db = all_db.reset_index(drop=True)

all_db[['InChI', 'InChIKey', 'SMILES', 'Chemical Formula', 'Mass', 'C']].info()
all_db.to_pickle('AllDB.pkl')

#### Now, classify compounds with the pybatchclassyfire (pybatchclassyfire.ipynb)

## Parse ClassyFire results

In [ ]:
#Flatten Intermediate Nodes and direct parent columns to respective level of classification
classyfire_df = pd.read_pickle('ClassyFire/cf_allraw.pkl')
classyfire_df = classyfire_df[['inchikey', 'smiles', 'kingdom.name', 'superclass.name', 'class.name', 'subclass.name', 
                                 'intermediate_nodes', 'direct_parent.name']]

classyfire_df['inchikey'] = classyfire_df['inchikey'].str.replace('InChIKey=', '')
classyfire_df = classyfire_df.rename(columns={'inchikey':'InChIKey', 'smiles':'SMILES', 'kingdom.name':'Kingdom', 
                                              'superclass.name':'Superclass', 'class.name': 'Class', 
                                              'subclass.name':'Subclass', 'intermediate_nodes': 'Intermediate Nodes',
                                              'direct_parent.name': 'Direct Parent'})

i_nodes_l = classyfire_df["Intermediate Nodes"].to_list()
i_nodes = pd.json_normalize(i_nodes_l)

def get_name(x):
    if x is not None:
        return x.get('name')
    else:
        return np.nan

columns = {i: 'Level ' + str(i+5) for i in i_nodes.columns}
columns_l = list(i_nodes.columns)
next_col = 'Level ' + str(columns_l[-1] + 6)
i_nodes = i_nodes.rename(columns=columns)

for level in columns.values():
    i_nodes[level] = i_nodes[level].apply(get_name)

classyfire_df = pd.concat([classyfire_df, i_nodes], axis=1).drop(columns='Intermediate Nodes')
classyfire_df[next_col] = np.nan
classyfire_df[next_col] = classyfire_df[next_col].astype('object')

levels = list(columns.values())
levels.append(next_col)
columns = ['Kingdom', 'Superclass', 'Class', 'Subclass'] + levels
    
for i, row in tqdm(classyfire_df.iterrows()):
    dir_p = row['Direct Parent']
    if dir_p in row[columns].values:
        continue
    else:
        for level, name in row[columns].iteritems():
            if type(name) is not float:
                continue
            else:
                classyfire_df.at[i, level] = dir_p
                break
classyfire_df = classyfire_df.drop(columns='Direct Parent')
classyfire_df.to_pickle('ClassyFire/cf_parsed.pkl')
classyfire_df.info()

## Merge ClassyFire output with the Databases

In [ ]:
all_db = pd.read_pickle('AllDB.pkl')
classyfire_df = pd.read_pickle('ClassyFire/cf_parsed.pkl')
classyfire_df.drop(columns='SMILES', inplace=True)
dataset = pd.merge(all_db, classyfire_df, on='InChIKey', how='left')

In [ ]:
dataset.info()

In [ ]:
# Take out compounds without classification
dataset = dataset[dataset['Kingdom'].notna()].reset_index(drop=True)

In [ ]:
#Remove columns from final Dataset and order the rest

columns = ['InChIKey', 'Kingdom', 'Superclass', 'Class', 'Subclass', 'Level 5', 'Level 6', 'Level 7', 'Level 8', 
           'Level 9', 'Level 10', 'Level 11', 'C', 'H', 'O', 'P', 'N', 'S', 'Th', 'Cm', 'K', 'Na', 
           'Rb', 'Li', 'Cs', 'Fr', 'Ca', 'Mg', 'Be', 'Sr', 'Ba', 'Ce', 'La', 'Nd', 'Gd', 'Sm', 'Eu', 'Lu', 'Pr', 'Tb', 
           'Dy', 'Ho', 'Er', 'Tm', 'Te', 'Si', 'As', 'B', 'Ge', 'Sb', 'Al', 'Bi', 'Cr', 'Co', 'Cu', 'Fe', 'Mo', 'Mn', 
           'Zr', 'Ti', 'W', 'Ni', 'V', 'Ag', 'Hg', 'Cd', 'Au', 'Ta', 'Y', 'Ru', 'Pd', 'Pt', 'Re', 'Zn', 'Hf', 'Nb', 'Sc', 
           'Os', 'Ir', 'Cl', 'I', 'F', 'Br', 'He', 'Ar', 'Se', 'Ga', 'Sn', 'Tl', 'Pb', 'Xe', 'Rn', 'Ra', 'Pu', 'Kr', 
           'U', 'Tc', 'At', 'In', 'Po', 'Ne', 'Ac', 'Rf', 'Db', 'Sg', 'Bh', 'Hs', 'Rh', 'Mt', 'Ds', 'Rg', 'Pm', 'Yb', 
           'Pa', 'Np', 'Am', 'Bk', 'Cf', 'Es', 'Fm', 'Md', 'No', 'Lr', 'Cn', 'Fl', 'Lv', 'Mc', 'Nh', 'Ts', 'Charge', 'Mass']
#Dropped ['InChI', 'SMILES', 'Chemical Formula']

dataset = dataset[columns]
dataset.to_pickle('Dataset.pkl')
dataset.info()